<a href="https://colab.research.google.com/github/bikash119/learn_tensorflow/blob/main/learn_tf_008.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning with Tensorflow
**Transfer Learning**: Taking the patterns (also called weights) another model has learned from another problem and using them for our own problem.
_Advantages_
1. Can leverage the architecture of the model proven to work on similar problem as ours.
2. Can leverage the working neural network architecture which has already learned patterns on data similar to our own. This results in achieving great results.

### What we will cover
1. Introduce **Transfer Learning**
2. Using a smaller dataset to experiment faster
3. Build a transfer learning feature extraction model using Tensorflow hub.
4. Introduce a **TensorBoard** callback to track model training results.
5. Compare model results using TensorBoard

In [1]:
# Add datetime
import datetime
print(f" Notebook last run end-to-end: {datetime.datetime.now()}")

 Notebook last run end-to-end: 2023-07-31 23:08:26.736346


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## Transfer learning with Tensorflow hub: Getting great results with 10% data

### Downloading and becoming one with data

In [3]:
# Get data ( 10% label)
import zipfile

#Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("/content/10_food_classes_10_percent.zip","r")
zip_ref.extractall()
zip_ref.close()

--2023-07-31 23:30:52--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.128, 74.125.197.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   157MB/s    in 1.0s    

2023-07-31 23:30:54 (157 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:
!rm -r /content/__MACOSX

In [8]:
# How many images in each folder
import os

for dirpath,dirname,filenames in os.walk("/content/10_food_classes_10_percent"):
  print(f" There are {len(dirname)} directories and {len(filenames)} images at location :{dirpath}")

 There are 2 directories and 0 images at location :/content/10_food_classes_10_percent
 There are 10 directories and 0 images at location :/content/10_food_classes_10_percent/test
 There are 0 directories and 250 images at location :/content/10_food_classes_10_percent/test/ice_cream
 There are 0 directories and 250 images at location :/content/10_food_classes_10_percent/test/pizza
 There are 0 directories and 250 images at location :/content/10_food_classes_10_percent/test/steak
 There are 0 directories and 250 images at location :/content/10_food_classes_10_percent/test/chicken_wings
 There are 0 directories and 250 images at location :/content/10_food_classes_10_percent/test/fried_rice
 There are 0 directories and 250 images at location :/content/10_food_classes_10_percent/test/chicken_curry
 There are 0 directories and 250 images at location :/content/10_food_classes_10_percent/test/hamburger
 There are 0 directories and 250 images at location :/content/10_food_classes_10_percent/te

### Creating Dataloaders (preparing the data)


In [10]:
# setup data inputs
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE= (224,224)
BATCH_SIZE= 32

train_dir= "/content/10_food_classes_10_percent/train"
test_dir= "/content/10_food_classes_10_percent/test"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print(" Training images.")
train_10_percent_data= train_datagen.flow_from_directory(train_dir
                                                         ,target_size=IMAGE_SHAPE
                                                         ,batch_size=BATCH_SIZE
                                                         ,class_mode="categorical")

print(" Test images")
test_10_percent_data= test_datagen.flow_from_directory(test_dir
                                                       ,target_size=IMAGE_SHAPE
                                                       ,batch_size=BATCH_SIZE
                                                       ,class_mode="categorical")


 Training images.
Found 750 images belonging to 10 classes.
 Test images
Found 2500 images belonging to 10 classes.


### Setting up the callbacks
**[Callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks)** are extra functionality that can be added to models which can be executed during or after or start of the training.

We will see how to use Tensorboard callback to track experiments. The Tensorboard callback can be accessed using `tf.keras.callbacks.Tensorboard()`
Its main functionality is saving model's training performance metrics to a specified `log_dir`

In [14]:
import tensorflow as tf
SAVE_DIR="model_logs"

In [15]:
# create tensorboard callback
import tensorflow as tf

def create_tensorboard_callback(dir_name,experiment_name):
  log_dir = dir_name+"/"+experiment_name+"/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

  return tf.keras.callbacks.Tensorboard(log_dir)

## Creating models using [Tensorflow hub](https://www.tensorflow.org/hub)

We will use the following models from the hub:
1. ResNetV2- 2016 state of the art computer vision model architecture.
2. EfficientNet: 2019 state of the art computer vision model architecture.

1. **As is, Transfer Learning** is when we take pretrained model as it is and apply it to our task without any changes.
  Say, we have problem where we want to classify the image across 1000 different classes of images, we can use this kind of Transfer Learning.Models with `/classification` in their name on Tensorflow hub provide this kind of functionality
2. **Feature Extraction Transfer Learning** is when we take underlying patterns of a pretrained model and adjust its output to fit our problem statement.
  For example, we have images which can be classified across 10 classes and we pick up a model which has been trained on ImageNet and can classify a image across 1000 classes. In such scenario, we just train few upper layers by changing the original activation layer which resembles the number of classes suitable to our problem. The important part is **only a few top layers become trainable, rest remains frozen**.
3. **Fine-tune transfer Learning** is when we take the underlying patterns (also called weights) and adjust to our problem statement. This usually means training **some, many or all** of the layers in the model.